In [1]:
%%capture --no-stderr
%pip install -qU langgraph
%pip install -qU langchain_ollama
%pip install -qU typing_extensions
%pip install -qU requests
%pip install -qU panel
%pip install -qU jupyter_bokeh
%pip install -qU html2text

In [16]:
# Step x. define the state
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [17]:
# Step x. define the graph builder
from langgraph.graph import StateGraph
graph_builder = StateGraph(State)

In [18]:
# Step x. define the model
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
)

In [19]:
# Step x. define the chatbot
def coding_assistant(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
# Step x. add the node to the graph
graph_builder.add_node("coding_assistant", coding_assistant)

In [ ]:
# Step x. add the start and end nodes
from langgraph.graph import START, END
graph_builder.add_edge(START, "coding_assistant")
graph_builder.add_edge("coding_assistant", END)

In [22]:
# Step x. compile the graph
graph = graph_builder.compile()

In [ ]:
# Step x. display the graph image
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [24]:
# Step x. define the stream function
from langchain_core.messages import HumanMessage
def stream_graph_updates(user_input: str):
    result = graph.invoke({"messages": [HumanMessage(content=user_input)]})
    return result.get("messages", [])[-1].content

In [ ]:
import panel as pn  # GUI
pn.extension()
panels = [] # collect display

In [26]:
inputStyles = {
    'color': '#666',
    'font-size': '16px',
    'border-radius': '5px',
    'padding': '2px 2px',
    'margin': '0px 10px 0px 5px',
}

prompt_input = pn.widgets.TextInput(value="", placeholder='Enter text here…', width=600, styles=inputStyles)
chat_button = pn.widgets.Button(name="Chat!", styles=inputStyles)

In [27]:
userStyles = {
    'color': '#ffa8db',
    'font-size': '14px',
    'font-family': 'Inter, Arial, sans-serif',
    'background-color': '#333', 'border': '1px solid #999',
    'border-radius': '5px', 
    'padding': '2px 10px',
    'width': '90%'
}

assistantStyles = {
    'color': '#11fa00',
    'font-size': '14px',
    'font-family': 'Inter, Arial, sans-serif',
    'background-color': '#333', 'border': '1px solid #999',
    'border-radius': '10px', 'padding': '2px 10px',
    'width': '90%'
}

import html
def escape_html(text):
    return html.escape(text)

def collect_messages(_):
    prompt = prompt_input.value_input
    if prompt != "":
        response = stream_graph_updates(prompt)
        panels.append(
            pn.Row('User:', pn.pane.HTML("<p>" + escape_html(prompt) + "</p>", width=600, styles=userStyles)))
        panels.append(
            pn.Row('Assistant:', pn.pane.Markdown(response, styles=assistantStyles)))
        prompt_input.value_input = ""
    return pn.Column(*panels)

In [ ]:
interactive_conversation = pn.bind(collect_messages, chat_button)
dashboard = pn.Column(
    pn.Row(prompt_input, chat_button),
    pn.Row(interactive_conversation),
)
dashboard